# Using mit-bih-noise-stress-test-database, Test our models and index performances

## Import

In [ ]:
from petastorm import make_reader
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
import os
import sys
sys.path.append(os.path.join(os.getcwd(), ".."))
from operations.dataset_manager import get_path_petastorm_format

## Set matplotlib window size and parameter

In [ ]:
plt.rcParams.update({"font.size": 32})
plt.rcParams["legend.fontsize"] = 32

## Paths

In [ ]:
models_save_path = "/workspaces/ecg_evaluation/results"
data_save_path = get_path_petastorm_format("mit-bih-noise-stress-test-database-1.0.0","ParquetFile")

# Get the dataset

In [ ]:
data = []
with make_reader(data_save_path) as reader:
    for row in reader:
        data.append(row)

In [ ]:
## get save files (where model is saved)

model_name = [
                f.split(".")[0]
                for f in os.listdir(models_save_path)
                if os.path.isfile(os.path.join(models_save_path,f )) and (f.endswith(".sav"))
            ]

In [ ]:
name_index = [
        "Corr_interlead",
        "Corr_intralead",
        "wPMF",
        "SNRECG",
        "HR",
        "Flatline",
        "TSD",
    ]

# Test the model on new dataset

## Divide into the 3 noises present in the dataset (em,ma,bw)

In [ ]:
em_data = []
ma_data = []
bw_data = []

for d in data:
    if d.noun_id.decode("utf-8") == "ma" or d.noun_id.decode("utf-8") == "oldma":
        ma_data.append(d)
    elif d.noun_id.decode("utf-8")== "em" or d.noun_id.decode("utf-8") == "oldem":
        em_data.append(d)
    elif d.noun_id.decode("utf-8") == "bw" or d.noun_id.decode("utf-8") == "oldbw":
        bw_data.append(d)

print(len(em_data))
print(len(ma_data))
print(len(bw_data))

## Test each model performance for each noise